In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

c:\Users\SALAH\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


100%|██████████| 26421880/26421880 [00:08<00:00, 3273534.25it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 406654.17it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3167586.85it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Inside the training loop, optimization happens in three steps:

1. **Reset Gradients**: Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

2. **Backpropagation**: Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.

3. **Parameter Update**: Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.


In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Full Implementation**

We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model’s performance against our test data.


In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.307289  [   64/60000]
loss: 2.291691  [ 6464/60000]
loss: 2.275367  [12864/60000]
loss: 2.269310  [19264/60000]
loss: 2.241053  [25664/60000]
loss: 2.212889  [32064/60000]
loss: 2.226881  [38464/60000]
loss: 2.188827  [44864/60000]
loss: 2.185959  [51264/60000]
loss: 2.159252  [57664/60000]
Test Error: 
 Accuracy: 40.2%, Avg loss: 2.147214 

Epoch 2
-------------------------------
loss: 2.159037  [   64/60000]
loss: 2.147186  [ 6464/60000]
loss: 2.085619  [12864/60000]
loss: 2.107976  [19264/60000]
loss: 2.047353  [25664/60000]
loss: 1.984676  [32064/60000]
loss: 2.021802  [38464/60000]
loss: 1.935583  [44864/60000]
loss: 1.939722  [51264/60000]
loss: 1.877452  [57664/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.865935 

Epoch 3
-------------------------------
loss: 1.898745  [   64/60000]
loss: 1.867866  [ 6464/60000]
loss: 1.742931  [12864/60000]
loss: 1.795684  [19264/60000]
loss: 1.686497  [25664/60000]
loss: 1.634117  [32064/600